In [1]:
import numpy as np
#import matplotlib.pyplot as plt

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout # GRU, Bidirectional
#from keras.optimizer import SGD
import math
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv(r"C:\Users\raksh\Downloads\GOOGL_2006-01-01_to_2018-01-01.csv", index_col='Date', parse_dates=['Date'])
data.head() #view first five columns

,Open,High,Low,Close,Volume,Name
Date,,,,,,
2006-01-03,211.47,218.05,209.32,217.83,13137450,GOOGL
2006-01-04,222.17,224.70,220.09,222.84,15292353,GOOGL
2006-01-05,223.22,226.00,220.97,225.85,10815661,GOOGL
2006-01-06,228.66,235.49,226.85,233.06,17759521,GOOGL
2006-01-09,233.44,236.94,230.70,233.68,12795837,GOOGL


In [3]:
mytrain = data[:'2016'].iloc[:,1:2].values #selecting all rows upto 2016 and selecting column 1
mytest = data['2017':].iloc[:,1:2].values #Tries to predict all rows after 2017 and selecting column 1

In [4]:
# Scaling the training set
sc = MinMaxScaler(feature_range=(0,1)) # MinMaxScaler(feature_range= (start,stop))
mytrain_scaled = sc.fit_transform(mytrain) #instance.fit_transform(data)

In [5]:
mytrain_scaled #view the scaled values!
#82.55 ==> 0.06065...

array([[0.11819421],
       [0.12763782],
       [0.12948394],
       ...,
       [0.96354625],
       [0.95278196],
       [0.94927433]])

In [6]:
len(mytrain_scaled)

2768

In [8]:
I_train = []
O_train = []
for i in range(60,2768):
    I_train.append(mytrain_scaled[i-60:i,0]) #every sequence will have 60 rows/values as input
    O_train.append(mytrain_scaled[i,0])

In [9]:
I_train[0]

array([0.11819421, 0.12763782, 0.12948394, 0.14296061, 0.14501974,
       0.14277599, 0.14622682, 0.14614161, 0.14039024, 0.14252038,
       0.1336164 , 0.13086143, 0.12130421, 0.11302508, 0.12479764,
       0.13138686, 0.12127581, 0.12001193, 0.1165043 , 0.12099179,
       0.09426567, 0.09746088, 0.0885143 , 0.08565992, 0.08125763,
       0.07201284, 0.07465421, 0.06761055, 0.05774092, 0.05850777,
       0.05446051, 0.06938567, 0.0730495 , 0.07404357, 0.07077736,
       0.07951092, 0.07868727, 0.0869522 , 0.09109887, 0.07113238,
       0.07941151, 0.08377119, 0.08104462, 0.07042234, 0.06444375,
       0.06336448, 0.05339544, 0.05453151, 0.0589906 , 0.058948  ,
       0.05642023, 0.05146411, 0.05737169, 0.05849357, 0.05311142,
       0.0542901 , 0.07158681, 0.07273708, 0.07711097, 0.09213553])

In [10]:
O_train[0]

0.08823028203016275

In [11]:
I_train = np.array(I_train) #converting into arrays!
O_train = np.array(O_train)

In [12]:
I_train.shape #samples,steps

(2708, 60)

In [14]:
I_train= I_train.reshape(2708,60,1)
#no. of sequences, inputs in every sequence, no. of features

In [15]:
I_train.shape

(2708, 60, 1)

In [16]:
#building the model

model = Sequential()
# first LSTM layer
model.add(LSTM(units=50, return_sequences=True, input_shape=(I_train.shape[1],1)))
model.add(Dropout(0.2))
# second LSTM layer
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# third LSTM layer
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# fourth LSTM layer
model.add(LSTM(units=50))
model.add(Dropout(0.2))
# the output layer
model.add(Dense(units=1))

C:\Users\raksh\anaconda4\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [17]:
#compile
model.compile(optimizer='rmsprop',loss='mean_squared_error')

In [18]:
model.fit(I_train,O_train,epochs=50,batch_size=32)

Epoch 1/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 11s 53ms/step - loss: 0.0308
Epoch 2/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - loss: 0.0062
Epoch 3/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 0.0049
Epoch 4/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.0048
Epoch 5/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 0.0046
Epoch 6/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0034
Epoch 7/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.0028
Epoch 8/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.0029
Epoch 9/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.0031
Epoch 10/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0026
Epoch 11/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 0.0027
Epoch 12/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.0027
Epoch 13/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 0.0023
Epoch 14/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.0022
Epoch 15/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.0019
Epo

In [19]:
model.save("StockGoogle.h5")

In [20]:
#test
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.preprocessing import MinMaxScaler
#load the model
model = load_model('StockGoogle.h5')
user_input = [83.58746017, 80.34122742, 80.81285462, 82.00291895, 83.29159961, 81.44688209, 80.76357678, 83.80165525, 82.51120409, 82.48569342, 80.65383659, 81.69740582, 82.78338862, 83.37704145, 83.53829805, 80.20374715, 80.42476455, 82.69989097, 83.73417143, 82.95666579, 83.94381372, 82.37856671, 83.60787851, 81.80850007, 82.61239819, 82.06021048, 81.0673498, 83.7835985, 81.74964384, 83.10783276, 80.54146548, 80.49215929, 82.03237911, 81.79802128, 80.93914228, 83.90444224, 83.2143073, 81.31844381, 83.23807452, 83.4603476, 81.85039395, 81.3031393, 80.6351211, 83.65161771, 80.54893536, 82.22940581, 82.50289209, 80.59368723, 83.24620825, 81.77694404, 83.20275117, 83.43996357, 83.38335394, 82.69282925, 80.80822433, 80.30044039, 83.6842126, 82.82960781, 80.12425038, 82.25607533]

user_input_array = np.array(user_input).reshape(-1, 1)
sc = MinMaxScaler(feature_range=(0,1))
user_scaled = sc.fit_transform(user_input_array)
user_scaled = user_scaled.reshape(1, 60, 1)

pred = model.predict(user_scaled)
print(pred)  # Changed pred(pred) to print(pred)
#up-scale
pred_original = sc.inverse_transform(pred)
print("The stock price is",pred_original[0][0]) #up scaled/original o/p

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 590ms/step
[[0.52883965]]
The stock price is 82.14419
